In [1]:
from syft_rds.orchestra import setup_rds_stack
from syft_rds import RDS_NOTEBOOKS_PATH

In [2]:
stack = setup_rds_stack(log_level="INFO")

2025-03-06 13:44:53.859 | WARNING  | syft_rds.orchestra:setup_rds_stack:89 - Using shared data directory for both clients. Any file permission checks will be skipped as both clients have access to the same files.
2025-03-06 13:44:53.864 | INFO     | syft_rds.orchestra:setup_rds_stack:112 - Launching mock RDS stack in /tmp/rds_xtkjmni3...


In [3]:
do_client = stack.do_rds_client
ds_client = stack.ds_rds_client

# Create a dataset as DO

In [4]:
private_path = RDS_NOTEBOOKS_PATH / "data" / "dataset-1" / "private"
mock_path = RDS_NOTEBOOKS_PATH / "data" / "dataset-1" / "mock"

private_path.mkdir(parents=True, exist_ok=True)
mock_path.mkdir(parents=True, exist_ok=True)

with open(RDS_NOTEBOOKS_PATH /"data/dataset-1/private/data.csv", "w") as f:
    f.write("-1,-2,-3")

with open(RDS_NOTEBOOKS_PATH / "data/dataset-1/mock/data.csv", "w") as f:
    f.write("1,2,3")


In [5]:
data = do_client.dataset.create(
    name="dataset-1",  # MUST BE UNIQUE. Throw Exception if already exist.
    path=private_path,  # MUST EXIST
    mock_path=mock_path,
)

In [6]:
do_client.datasets

[Dataset
   uid: abbd00fb-17c7-4a9b-a184-b420ff3426ed
   created_at: 2025-03-06T11:44:53.977108Z
   updated_at: 2025-03-06T11:44:53.977111Z
   name: dataset-1
   private: syft://data_owner@test.openmined.org/private/datasets/dataset-1
   mock: syft://data_owner@test.openmined.org/public/datasets/dataset-1
   summary: None
   readme: None
   tags: []]

# Use dataset as DS

In [7]:
ds_client.datasets

[Dataset
   uid: abbd00fb-17c7-4a9b-a184-b420ff3426ed
   created_at: 2025-03-06T11:44:53.977108Z
   updated_at: 2025-03-06T11:44:53.977111Z
   name: dataset-1
   private: syft://data_owner@test.openmined.org/private/datasets/dataset-1
   mock: syft://data_owner@test.openmined.org/public/datasets/dataset-1
   summary: None
   readme: None
   tags: []]

In [8]:
ds_client.dataset.get("dataset-1").get_mock_path()


PosixPath('/tmp/rds_xtkjmni3/shared_client_dir/datasites/data_owner@test.openmined.org/public/datasets/dataset-1')

In [9]:
ds_client.dataset.get("dataset-1").get_private_path()

PosixPath('/tmp/rds_xtkjmni3/shared_client_dir/datasites/data_owner@test.openmined.org/private/datasets/dataset-1')

In [10]:
dataset = ds_client.dataset.get("dataset-1")
dataset.describe()

└── data.csv


True

In [11]:
import pathlib
from syft_rds.models.models import Dataset
from syft_rds.utils.render_code_template import render_data_access_template

In [12]:
my_file = RDS_NOTEBOOKS_PATH / "data" / "code1.py"

In [13]:
%%writefile {my_file}

import os

DATA_DIR = os.environ["DATA_DIR"]
OUTPUT_DIR = os.environ["OUTPUT_DIR"]

print("Hello, world!")

with open(os.path.join(OUTPUT_DIR, "output.txt"), "w") as f:
    f.write("ABC")


Overwriting /home/teo/OpenMined/rds/notebooks/data/code1.py


In [14]:
dataset.name

'dataset-1'

In [15]:
job = ds_client.jobs.submit(
    user_code_path=my_file,
    dataset_name = dataset.name
)
job


Job
  uid: 0f5e661b-f146-4ece-ad5c-0fd0c759e564
  created_at: 2025-03-06T11:44:54.163568Z
  updated_at: 2025-03-06T11:44:54.163569Z
  name: virtual_packet_7zfn
  description: None
  runtime: python
  user_code_id: 4913376c-1bdc-4a75-bddc-6cce36cdb8f4
  tags: []
  user_metadata: {}
  status: pending_code_review
  error: no_error
  output_url: syft://data_owner@test.openmined.org/api_data/RDS/output/0f5e661b-f146-4ece-ad5c-0fd0c759e564
  dataset_name: dataset-1

In [16]:
ds_client.run_mock(job)

╭─────────────────────────────────────────────── Job Configuration ───────────────────────────────────────────────╮
│ Starting job                                                                                                    │
│ Execution: python code1.py                                                                                      │
│ Dataset:  /tmp/rds_xtkjmni3/shared_client_dir/datasites/data_owner@test.openmined.org/public/datasets/dataset-1 │
│ → /data                                                                                                         │
│ Output:   virtual_packet_7zfn/output → /output                                                                  │
│ Timeout:  1s                                                                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Running job...

Hello, world!

Job completed successfully!

Job
  uid: 0f5e661b-f146-4ece-ad5c-0fd0c759e564
  created_at: 2025-03-06T11:44:54.163568Z
  updated_at: 2025-03-06T11:44:54.163569Z
  name: virtual_packet_7zfn
  description: None
  runtime: python
  user_code_id: 4913376c-1bdc-4a75-bddc-6cce36cdb8f4
  tags: []
  user_metadata: {}
  status: pending_code_review
  error: no_error
  output_url: syft://data_owner@test.openmined.org/api_data/RDS/output/0f5e661b-f146-4ece-ad5c-0fd0c759e564
  dataset_name: dataset-1

In [17]:
do_job = do_client.jobs.get_all()[-1]

In [18]:
do_job.add_to_queue()

In [19]:
from syft_rds.client.utils import PathLike
from syft_rds.models.models import Job, JobStatus, JobUpdate


In [20]:
from syft_rds.models.models import JobErrorKind


do_client.rpc.jobs.update(
        JobUpdate(
            uid=do_job.uid,
            status=JobStatus.queued,
            error=JobErrorKind.no_error
        ))

Job
  uid: 0f5e661b-f146-4ece-ad5c-0fd0c759e564
  created_at: 2025-03-06T11:44:54.163568Z
  updated_at: 2025-03-06T11:44:54.446402Z
  name: virtual_packet_7zfn
  description: None
  runtime: python
  user_code_id: 4913376c-1bdc-4a75-bddc-6cce36cdb8f4
  tags: []
  user_metadata: {}
  status: queued
  error: no_error
  output_url: syft://data_owner@test.openmined.org/api_data/RDS/output/0f5e661b-f146-4ece-ad5c-0fd0c759e564
  dataset_name: dataset-1

In [21]:
do_client.jobs.get_all()

[Job
   uid: 0f5e661b-f146-4ece-ad5c-0fd0c759e564
   created_at: 2025-03-06T11:44:54.163568Z
   updated_at: 2025-03-06T11:44:54.446402Z
   name: virtual_packet_7zfn
   description: None
   runtime: python
   user_code_id: 4913376c-1bdc-4a75-bddc-6cce36cdb8f4
   tags: []
   user_metadata: {}
   status: queued
   error: no_error
   output_url: syft://data_owner@test.openmined.org/api_data/RDS/output/0f5e661b-f146-4ece-ad5c-0fd0c759e564
   dataset_name: dataset-1]

In [22]:
def share_results(self, job: Job, job_output_path: PathLike) -> Job:
    self.local_store.jobs.share_result_artifacts(job, job_output_path)

    job = self.rpc.jobs.update(
        JobUpdate(
            uid=job.uid,
            status=JobStatus.shared,
        )
    )
    return job

In [23]:
private_path = do_client.dataset.get(do_job.dataset_name).get_private_path()

In [24]:
do_job = do_client.run_private(do_job)

╭─────────────────────────────────────────────── Job Configuration ───────────────────────────────────────────────╮
│ Starting job                                                                                                    │
│ Execution: python code1.py                                                                                      │
│ Dataset:                                                                                                        │
│ /tmp/rds_xtkjmni3/shared_client_dir/datasites/data_owner@test.openmined.org/private/datasets/dataset-1 → /data  │
│ Output:   virtual_packet_7zfn/output → /output                                                                  │
│ Timeout:  1s                                                                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Running job...

Hello, world!

Job completed successfully!

In [25]:
do_job

Job
  uid: 0f5e661b-f146-4ece-ad5c-0fd0c759e564
  created_at: 2025-03-06T11:44:54.163568Z
  updated_at: 2025-03-06T11:44:54.812954Z
  name: virtual_packet_7zfn
  description: None
  runtime: python
  user_code_id: 4913376c-1bdc-4a75-bddc-6cce36cdb8f4
  tags: []
  user_metadata: {}
  status: job_run_finished
  error: no_error
  output_url: syft://data_owner@test.openmined.org/api_data/RDS/output/0f5e661b-f146-4ece-ad5c-0fd0c759e564
  dataset_name: dataset-1

In [29]:
do_client.jobs.share_results(do_job, RDS_NOTEBOOKS_PATH / "data" / "output")

Job
  uid: 0f5e661b-f146-4ece-ad5c-0fd0c759e564
  created_at: 2025-03-06T11:44:54.163568Z
  updated_at: 2025-03-06T11:45:30.181856Z
  name: virtual_packet_7zfn
  description: None
  runtime: python
  user_code_id: 4913376c-1bdc-4a75-bddc-6cce36cdb8f4
  tags: []
  user_metadata: {}
  status: shared
  error: no_error
  output_url: syft://data_owner@test.openmined.org/api_data/RDS/output/0f5e661b-f146-4ece-ad5c-0fd0c759e564
  dataset_name: dataset-1